In [1]:
from pprint import pprint as pp
import numpy as np
import cv2
import random as rnd
from pathlib import Path

In [6]:
class Line():
  def __init__(self, index, top, down) -> None:
    self.index = index
    self.down = down
    self.top = top

  def __getitem__(self, key):
    if key == 0: return self.index
    if key == 1: return self.top
    if key == 2: return self.down
    
  def __repr__(self):
    return ("{ index: " + str(self.index) + ", " +\
            "top: " + str(self.top) + ", "+ \
            "down: " + str(self.down) + "}")

def get_mask(img):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # define range of black color in HSV
    lower_val = np.array([0,0,0])
    upper_val = np.array([50,100,100])

    # Threshold the HSV image to get only black colors
    mask = cv2.inRange(hsv, lower_val, upper_val)

    # invert mask to get black symbols on white background
    mask_inv = cv2.bitwise_not(mask)

    return mask_inv

def get_mask_from_gray(img):
    # define range of black color in HSV
    lower_val = 0
    upper_val = 100

    # Threshold the HSV image to get only black colors
    mask = cv2.inRange(img, lower_val, upper_val)
    mask = cv2.bitwise_not(mask)

    return mask

def get_lines(mask_inv):
    graph = []

    for j in range(mask_inv.shape[1]):
        higher = 0
        lower = 0

        cntBlank = 0
        lines = []

        for i in range(mask_inv.shape[0]):
            if mask_inv[i,j] == 255: 
                cntBlank += 1
                if lower == 0:
                    continue
            
            if lower == 0: lower = i
            if mask_inv[i,j] != 255: 
                higher = i
                cntBlank = 0
            if cntBlank > 10:
                lines.append(Line(j, higher, lower))
                lower = 0
                higher = 0
        # if len(lines) > 0:
        for l in lines:
            graph.append(l)
        lines.clear()
    
    return graph

def draw_islands(ecg, mask_inv, path=r'../images/output/islands.png'):
    test_img = np.ones((
        mask_inv.shape[0],
        mask_inv.shape[1],
        3
        )) * 255

    for island in ecg:
        clr = (
            rnd.randint(0, 255),
            rnd.randint(0, 255),
            rnd.randint(0, 255)
        )
        for i in island:
            cv2.line(
                test_img, 
                (i.index, i.top),
                (i.index, i.down),
                clr
                )
    cv2.imwrite(path, test_img)

def get_low_up(graph, img=np.zeros(0)):

    prev = graph[0]
    lastAcc = prev[0]

    prevPoint = 0
    mainPoints = []
    mainPoints.append(graph[0].top)
    prevPoint = graph[0].down
    status = False
    shape = len(img.shape)
    print("Я начну с точки ", mainPoints[0])

    if shape > 1:
        if shape == 2:
            clr = 25
        else:
            clr = (0, 255, 0)
        img[mainPoints[0]][graph[0].index] = clr

    for p in graph[1:]:
            
        if(status == False):
            if(mainPoints[-1] < p.top):
                if(prevPoint <= p.down):
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
                else:
                    if((p.top - mainPoints[-1]) > (p.down - prevPoint)):
                        status = False
                        mainPoints.append(p.top)
                        prevPoint = p.down
                    elif((p.top - mainPoints[-1]) < (p.down - prevPoint)):
                        status = True
                        mainPoints.append(p.down)
                        prevPoint = p.top
                    else:
                        status = False
                        mainPoints.append(p.top)
                        prevPoint = p.down
            elif(mainPoints[-1] > p.top):
                if(prevPoint >= p.down):
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
                else:
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
            else:
                if(prevPoint <= p.down):
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
                else:
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
        else:
            if(mainPoints[-1] > p.down):
                if(prevPoint >= p.top):
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
                else:
                    if((p.down - mainPoints[-1]) < (p.top - prevPoint)):
                        status = True
                        mainPoints.append(p.down)
                        prevPoint = p.top
                    elif((p.down - mainPoints[-1]) > (p.top - prevPoint)):
                        status = False
                        mainPoints.append(p.top)
                        prevPoint = p.down
                    else:
                        status = True
                        mainPoints.append(p.down)
                        prevPoint = p.top
            elif(mainPoints[-1] < p.down):
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
            else:
                if(p.top > prevPoint):
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
                else:
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top

        if shape > 1:
            if shape == 2:
                clr = 200
            else:
                clr = (0, 255, 0)
            
            blank3 = np.ones(img.shape, np.uint8)*255
            cv2.line(img, (p[0],p[2]), (p[0], p[1]), clr)
            cv2.line(blank3, (p[0],p[2]), (p[0], p[1]), clr)
            
            if len(img.shape) == 2:
                clr = 255 if status else 0
            else:
                clr = (255, 255, 0) \
                    if status else (0, 255, 255)
            
            img[mainPoints[-1]][p.index] = clr

    if len(img.shape) > 1:
        cv2.imwrite(r'../images/output/output2.png',img)

    return mainPoints

In [33]:
img = cv2.imread(r"../images/input/input.jpg", cv2.IMREAD_GRAYSCALE)
img_clr = cv2.imread(r"../images/input/input.jpg")
mask_inv = get_mask_from_gray(img)

graph = get_lines(mask_inv)

ecg = []
print (len(graph))
while (len(graph) != 0):
    temp = [graph[0]]
    for k in graph[1:]:
        if (temp[-1].index != k.index-1):
            # если эта точка не сосед предыдущей
            continue
        
        # if (temp[-1].top > k.down or k.top < temp[-1].down):
        #     # если эта точка не сосед предыдущей
        #     continue
            
        # если эта точка все таки сосед предыдущей
        temp.append(k)
    
    if (len(temp) != 0):
        # print(temp[0])
        ecg.append(temp)
        for i in temp[::-1]:
            graph.remove(i)

draw_islands(ecg, mask_inv)
t = get_low_up(get_lines(mask_inv), img)

4661
{ index: 1, top: 19, down: 7}
{ index: 2, top: 19, down: 1}
{ index: 3, top: 19, down: 1}
{ index: 4, top: 18, down: 2}
{ index: 5, top: 20, down: 2}
{ index: 6, top: 20, down: 2}
{ index: 7, top: 20, down: 2}
{ index: 8, top: 20, down: 3}
{ index: 9, top: 20, down: 3}
{ index: 10, top: 18, down: 1}
{ index: 11, top: 18, down: 1}
{ index: 12, top: 19, down: 1}
{ index: 13, top: 19, down: 1}
{ index: 14, top: 19, down: 1}
{ index: 15, top: 17, down: 1}
{ index: 16, top: 21, down: 1}
{ index: 17, top: 21, down: 1}
{ index: 18, top: 17, down: 1}
{ index: 19, top: 17, down: 1}
{ index: 20, top: 19, down: 1}
{ index: 21, top: 19, down: 1}
{ index: 22, top: 20, down: 1}
{ index: 23, top: 21, down: 1}
{ index: 24, top: 20, down: 1}
{ index: 25, top: 20, down: 1}
{ index: 26, top: 20, down: 1}
{ index: 27, top: 21, down: 3}
{ index: 28, top: 20, down: 16}
{ index: 29, top: 20, down: 18}
{ index: 30, top: 20, down: 17}
{ index: 31, top: 20, down: 18}
{ index: 32, top: 21, down: 17}
{ index

In [4]:
cv2.imwrite(r'../images/output/mask_inv.png',mask_inv)
cv2.imwrite(r'../images/output/output2.png',img)
# cv2.imwrite(r'../images/output/output.png',blank3)

False